## Notebook to import the master whisky list

In [ ]:
# Importing all the necessary libraries for this project.

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import json
import time

In [ ]:
# VIEW ALL COLUMNS & ALL TEXT IN EACH CELL

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)

In [ ]:
## Defining the function to request the website and return the soup

def beautiful_soup(url):
    request = requests.get(url)
    soup = BeautifulSoup(request.text, "lxml")
    return soup


In [ ]:
## Scraping the list of bottles

def scraper_info(soup, newurl):
    for posts in soup.find_all('div' , {'class' : 'group group-list'}):
        
# Bottle Name        
        try:
            for bottle in posts.find_all('div', {'class' : 'name'}):
                bottle_name.append(str(bottle).split("<span>")[0].replace('<div class="name">','').split('<')[0].lstrip())
        except:
                bottle_name.append('NoName')
# Bottle Link        
        try:       
            for link in posts.find_all('a'):
                bottle_link.append("https://www.thewhiskyexchange.com" + link.get('href'))
        except:
            bottle_link.append(np.NaN)

#Volume & ABV            
        try:    
            for ml in posts.find_all('span', {'class' : 'meta'}):
                volume.append(ml.text.split("/")[0].rstrip())
                abv.append(ml.text.split("/")[1].lstrip())
                whisky_class(newurl)

        except:
                volume.append(np.NaN)
                abv.append(np.NaN)

#Price                
        try:
            for pricing in posts.find_all('span', {'class' : 'price'}):
                prices.append(pricing.text)
        except:
                prices.append(np.NaN)

In [ ]:
## Using the sitemap, listing all the major urls to scrape from

list_url = ['https://www.thewhiskyexchange.com/c/40/single-malt-scotch-whisky','https://www.thewhiskyexchange.com/c/304/blended-scotch-whisky',
           'https://www.thewhiskyexchange.com/c/309/blended-malt-scotch-whisky', 'https://www.thewhiskyexchange.com/c/310/grain-scotch-whisky',
           'https://www.thewhiskyexchange.com/d/677/organic-whisky', 'https://www.thewhiskyexchange.com/c/32/irish-whiskey', 
            'https://www.thewhiskyexchange.com/c/33/american-whiskey', 'https://www.thewhiskyexchange.com/c/34/canadian-whisky',
           'https://www.thewhiskyexchange.com/c/35/japanese-whisky', 'https://www.thewhiskyexchange.com/c/305/rest-of-the-world-whisky']

In [ ]:
## LIST OF DIFFERENT TYPES OF WHISKY

def whisky_class(url):
    if 'single-malt-scotch-whisky' in url:
        whiskytype.append("Single Malt Scotch")
    elif 'blended-scotch-whisky' in url:
        whiskytype.append("Blended Scotch")
    elif 'blended-malt-scotch-whisky' in url:
        whiskytype.append("Blended Malt Scotch")
    elif 'grain-scotch-whisky' in url:
        whiskytype.append("Grain Scotch")
    elif 'organic-whisky' in url:
        whiskytype.append("Organic Scotch")
    elif 'irish-whiskey' in url:
        whiskytype.append("Irish Whisky")
    elif 'american-whiskey' in url:
        whiskytype.append("American Whisky")
    elif 'canadian-whisky' in url:
        whiskytype.append("Canadian Whisky")
    elif 'japanese-whisky' in url:
        whiskytype.append("Japanese Whisky")
    elif 'rest-of-the-world-whisky' in url:
        whiskytype.append("Rest of World Whisky")
    else:
        whiskytype.append(np.NaN)


In [ ]:
%%time

# DEFINE THE EMPTY LISTS HERE
bottle_name = []
bottle_link = []
volume = []
abv = []
distillery = []
prices = []
whiskytype = []


# START A COUNTER & BLANK URL/

for url in list_url:
    count = 1
    running = True
    while running == True:

    # SET A COUNTER = TO NOT OVERLOAD THE PAGES

        if (count%50 == 0):
            time.sleep(5)
            print(str(count) + " pages have been scraped")

    ## URL CHECK
        newurl = url + '?filter=true&pg=' + str(count) + '#productlist-filter'
        soup = beautiful_soup(newurl)
        for pagecheck in soup.find_all('div', {'class' : 'showing'}):
            last = pagecheck.text.split()[-1]
            limit = pagecheck.text.split()[-3]
            if last == limit:
                running = False
            scraper_info(soup, newurl)
            count = count + 1        


In [ ]:
## EXPORTING THE MAIN SITE TYPE HERE.

whisky_pd = pd.DataFrame({"BottleName" : bottle_name, "Link" : bottle_link, "Volume": volume, 
                           "ABV" : abv, "Price/Bottle": prices, 'WhiskyType' : whiskytype})

### EXPORT TO CSV

In [ ]:
whisky_pd.to_csv("Whisky_Main.csv", header=True)

In [ ]:
## DEFINING THE INFO TABLE HERE

def infocrumb(soup):

## Get the description of the bottle
    try:
        description.append(soup.find('div', itemprop = 'description').text.lstrip().rstrip().replace('\n', ''))
    except:
        description.append(np.NaN)
    
    
## MAKE CASES FOR EACH TYPE OF WHISKEY    
    if soup.find('dl', {'class' : 'meta'}):
        for desc in soup.find_all('dl', {'class' : 'meta'}):
            keys = []
            values = []
            for iterdt in desc.find_all('dt'):
                keys.append(iterdt.text)

            for iterdd in desc.find_all('dd'):
                values.append(iterdd.text)

            dictionary = dict(zip(keys, values))
            details.append(dictionary)
    else:
        details.append(np.NaN)
        
    if soup.find('div', {'class' : 'name-container'}):  
        for iternname in soup.find_all('div', {'class' : 'name-container'}):
            whisky_name.append(iternname.h1.text.lstrip('\n').rstrip('\n').replace("\n\n", " "))
    else:
        for desc in soup.find_all('div', {'class': 'breadcrumb'}):
            whisky_name.append(desc.find_all("li")[-1].find("span").text)
        
    
    if soup.find('div', {'class' : 'rating-overview'}):
        for iterrat in soup.find_all('div', {'class' : 'rating-overview'}):
            ratings.append(float(iterrat.text.splitlines()[4]))
            reviews.append(int(iterrat.text.split('(')[1].split()[0]))
    else:
        ratings.append(np.NaN)
        reviews.append(np.NaN)
        
    if soup.find_all('div', {'class' : 'flavour'}):
        newflav = ''
        for iterflav in soup.find_all('div', {'class': 'flavour'}):
            newflav = newflav + iterflav.text.replace('\n', ' ').lstrip().rstrip()
        character.append(newflav)
    else:
        character.append(np.NaN)
        
    for desc in soup.find_all('div', {'class': 'breadcrumb'}):
        distillery.append(desc.find_all("li")[-2].find("span").text)

In [ ]:
### DEFINING THE CRUMBS HERE FOR REVIEW

def reviewcrumb(soup, url):
    reviewrating = {}
    if soup.find_all('ul', {'class' : 'reviews-list'}):
        keys = []
        values = []
        for rev in (soup.find_all('span', {'itemprop': 'ratingValue'})):
            values.append(rev.text)
        for rat in (soup.find_all('p', {'itemprop': 'reviewBody'})):
            keys.append(rat.text)
        if len(keys)!=len(values):
            del values[0]
        reviewrating[url]=dict(zip(keys, values))
    else:
        reviewrating[url]={}
    
    review_rating.append(reviewrating)

In [ ]:
## Starting the scrape of each individual whisky

description = []
details = []
whisky_name = []
ratings = []
reviews = []
revnotes = []
character = []
distillery = []
whiskyurl = []
review_rating = []


def whisky_scraper(url):
    
    ''' DOCSTRING: The main function for scraping individual whisky pages''' 
    
    ## Creating an empty list and adding the urls of each individual whisky here. Similar to whisky_pd['Links']
    
    whiskyurl.append(url)
    
    ## Calling the beautiful soup function here
    soup = beautiful_soup(url)

    ## Calling the page scraper Function
    infocrumb(soup)
    
    
    ## Creating a dictionary of each soup here.
    reviewcrumb(soup, url)


### RUN THE SCRAPER HERE! TAKES ~19 mins to scrape entire website

In [ ]:
%%time

whisky_pd['Link'].apply(whisky_scraper)

In [ ]:
whisky_details = pd.DataFrame({"Description" : description, "Details" : details, "Whisky": whisky_name, 
                           "#Ratings" : ratings, "#Reviews": reviews, "Character": character, "Distillery": distillery, 
                              "WhiskyURL" : whiskyurl})


In [ ]:
whisky_details = whisky_details.join(whisky_details['Details'].apply(pd.Series))

### EXPORT WHISKY DETAILS HERE

In [ ]:
whisky_details.to_csv('Whisky Details.csv', header=True)

### CREATING A FUNCTION TO EXPLORE THE UNWRAPPING OF THE DICTIONARY. 

-- Up until now the whisky reviews have been stored individually. The format of the dictionary of whisky reviews is as follows: ${url : {rating:review}}$, where url is a unique entry and rating/review changes from item to item.

In [ ]:
final_review = pd.DataFrame(index=None, columns=['Link', 'Ratings', 'Reviews'])

for url in review_rating:
    pd_reviews = pd.DataFrame.from_dict(url,orient = 'columns').reset_index()
    pd_reviews['Link']= pd_reviews.columns[1]
    link = pd_reviews.columns[1]
    pd_reviews = pd_reviews.rename(columns={link: 'Ratings', 'index': 'Reviews'})
    pd_reviews = pd_reviews[['Link', 'Ratings', 'Reviews']]
    final_review = pd.concat([final_review, pd_reviews])

In [ ]:
final_review.to_csv('Final_Review.csv', header=True)

In [ ]:
## IMPORTING DISTILLERY INFORMATION & SAVING IT AS A CSV

In [ ]:
## Importing the other files. From Wikipedia - https://en.wikipedia.org/wiki/List_of_whisky_distilleries_in_Scotland

distillery_main = pd.read_csv("ScotlandDistilleryList.csv", )
distillery_main['Status']='Active'

distillery_silent = pd.read_csv("SilentDistillery.csv")
distillery_silent['Status']='Silent'

distillery_grain = pd.read_csv("GrainDistilleryList.csv")
distillery_grain['Status']='Active'

distillery_silentg = pd.read_csv("SilentGrainList.csv")
distillery_silentg['Status']='Silent'

In [ ]:
distillery_silent

In [ ]:
## Combining all Distillery information into one

frames = [distillery_main, distillery_silent, distillery_silent, distillery_grain]
distillery_main = pd.concat(frames).reset_index(drop = True)

distillery_main['Distillery'] = distillery_main['Distillery'].apply(lambda x: (x.replace(' distillery', '')))

distillery_main

In [ ]:
distillery_main.to_csv('Distillery_Info.csv', header=True)